In [ ]:
import json
import requests
from dateutil import parser

#downloading page
url = "https://factorio.com/galaxy/"

response = requests.get(url)

if response.status_code != 200:

    raise Exception(f"Failed to download the webpage. Status code: {response.status_code}")

#extracting data
data_start = response.text.find('initSpacemapViewer(') + len('initSpacemapViewer(')

data_end = response.text.find(']}}') + len(']}}')

data = json.loads(response.text[data_start:data_end])

#seperating dates. dates array has creation date and update date for each user
creation_dates= []
update_dates = []
for index,date in enumerate(data["stars"]['creation_update']):
    if not index%2:
        creation_dates.append(date)
        # print(index,date)
    else:
         update_dates.append(date)

#loop on sorted creation_dates list and saving all data in json
sorted_users=[]
for date in sorted(creation_dates):
        index = data["stars"]["creation_update"].index(date)//2
        user = data["stars"]['users'][index]
        if user == '': user = '[empty]'
        update_date = update_dates[creation_dates.index(date)]
        galaxy_name = data["stars"]['names'][index]
        id = data["stars"]['ids'][index]

        python_time = parser.parse(date)
        creation_date_js = python_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

        python_time = parser.parse(update_date)
        update_date_js = python_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        sorted_users.append(
            {'user':user,
            'creation_date':creation_date_js,
            'update_date':update_date_js,
            'galaxy_name':galaxy_name,
            'id':id}
            )

output_file = "data.json"

with open(output_file, "w") as file:
    json.dump({"data": sorted_users}, file,indent=2)